In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import gym
import numpy
import onnx
import onnxruntime
from gym import wrappers
import os
from typing import Any, Dict, List, Optional
import glob

import argparse
import os
import time
#import caffe2.python.onnx.backend as onnx_caffe2

# import geffnet
# from base.rl.ppo import PPO

pthfile = 'pth/net_params.pth'
onnxpath = 'onnx/net.onnx'
print(torch.__version__)
print(onnx.__version__)
print(onnxruntime.__version__)

1.2.0
1.6.0
1.6.0


In [2]:
class Flatten(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x.view(x.shape[0], -1)

class Global_Actor(nn.Module):
    def __init__(self):
        super().__init__()
        self.actor = nn.Sequential(  # (3, 1200, 1200)
            nn.Conv2d(3, 8, 5, padding=2),  # (8, 1200, 1200)
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # (8, 600, 600)
            nn.Conv2d(8, 16, 3, padding=1),  # (16, 600, 600)
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # (16, 300, 300)
            nn.Conv2d(16, 32, 5, padding=2),  # (32, 300, 300)
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # (64, 150, 150)
            nn.Conv2d(32, 64, 3, padding=1),  # (64, 150, 150)
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 1, 1, padding=0),  # (1, 150, 150)
            Flatten(),  # (G*G, )
            # nn.Sigmoid(),  # frontier_mask
        )
        
    def forward(self, inputs):
        x2 = self.actor(inputs)
        return x2

In [7]:
def test():
    M = 1200
    batch_size = 1
    input_shape = (3, M, M)
    
    # Initialize model with the pretrained weights
    # map_location = 'gpu' if torch.cuda.is_available() else 'cpu'
    # loaded_model = torch.load(pthfile, map_location="cpu")
    # model.actor.load_state_dict(loaded_model, strict=False)
    
    model = Global_Actor()
    print(model.actor)
    model.actor.load_state_dict(torch.load(pthfile))
    # set the model to inference mode
    model.eval()
    
    inputs = torch.randn(1, 3, 1200, 1200)
    map3 = torch.zeros(1, 1200, 1200)
    map3[0][212][200] = 1.
    inputs[0][2] = map3
    x = inputs
    # data type nchw
    # x = torch.rand(batch_size, *input_shape)
    input_names = ["input0"]
    output_names = ["output0"]
    optional_args = dict(keep_initializers_as_inputs=True)  # pytorch 1.3 needs this for export to succeed

    try:
        torch_out = torch.onnx._export(
            model, x, onnxpath, export_params=True, verbose=False,
            input_names=input_names, output_names=output_names, **optional_args,
            opset_version=10)
    except TypeError:
        # fallback to no keep_initializers arg for pytorch < 1.3
        torch_out = torch.onnx._export(
            model, x, onnxpath, export_params=True, verbose=False,
            input_names=input_names, output_names=output_names, opset_version=10)
    '''
    # # Export the model
    torch.onnx.export(model, x, 
                      onnxpath, 
                      opset_version=10)
                      #keep_initializers_as_inputs=True, 
                      # verbose=True
     '''
    print("=============Successful==========")

In [8]:
test()

Sequential(
  (0): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (9): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU()
  (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (12): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (14): ReLU()
  (15): Conv2d(64, 1, kernel_size=(1, 1), str